In [1]:
import pickle
import seaborn as sns

In [2]:
# load pickle files before append new path to the sys.path for loanding custom modules
loaded_model = pickle.load(open(r'N:\10. Executive\Original\3. 2019\Gender Classifier\gender_classifier.pkl', 'rb'))
loaded_vectorizer = pickle.load(open(r'N:\10. Executive\Original\3. 2019\Gender Classifier\vectorizer.pkl', 'rb'))

In [3]:
# Add path in order to load utility functions
import sys
sys.path.append(r"N:\18. Hot Spot\Custom Modules\Utils")
sys.path.append(r"N:\18. Hot Spot\Custom Modules\ETLMonthly")

In [4]:
# setup environment and import libraries
import os
import re
import datetime
import time
import getpass
import pandas as pd
import numpy as np
from datetime import datetime, date
from datetime import timedelta
import calendar as cd
from dateutil.relativedelta import relativedelta
from collections import OrderedDict
path=r'C:\ProgramData\Anaconda2\instantclient_12_2'
os.environ['PATH']=path
import cx_Oracle
from sqlalchemy import create_engine
import sqlalchemy
import file_utils
import db_utils
import modeling_utils
import config

#set working directory
os.chdir(config.working_dir)

In [5]:
# db credentials authentication

In [6]:
print 'connect to database...'
conn_string = db_utils.db_connect(config.db_username,config.db_pwd)

connect to database...
Enter username: data_analyst
Enter Password:········
Press 'y' to continue, 'c' to cancel or escape:y
Testing Connection...
Connection Succesful!


### Import Data

In [7]:
con = cx_Oracle.connect(conn_string)
cs_retail_loan_cust_with_names = pd.read_sql_query('''
select a.bra_code, a.cus_num,
(case when cus_name like '%XXX%' then sec_cus_sho_name else cus_name end) cus_name
from cs_retail_loan_cust a, stg.src_customer_extd@exadata_new b
where a.bra_code=b.bra_code and a.cus_num=b.cus_num
''', con)
con.close()

In [8]:
df_loan_data_basic_var = pd.read_csv('df_loan_data_basic_var.csv').merge(
        cs_retail_loan_cust_with_names, on=['BRA_CODE','CUS_NUM'], how='left')

In [9]:
# Convert column types
df_loan_data_basic_var['DATE_OPEN'] = pd.to_datetime(df_loan_data_basic_var['DATE_OPEN'])
df_loan_data_basic_var['BIR_DATE'] = pd.to_datetime(df_loan_data_basic_var['BIR_DATE'])
df_loan_data_basic_var['CUS_DATE_OPEN'] = pd.to_datetime(df_loan_data_basic_var['CUS_DATE_OPEN'])

In [10]:
# df_loan_data_basic_var[df_loan_data_basic_var['AIRTIME_NBR_MONTH_ACTIVE_RATIO']>1][[
#     'CUS_DATE_OPEN','DATE_OPEN','AIRTIME_NBR_MONTH_ACTIVE_RATIO','AIRTIME_NBR_MONTH_ACTIVE','BANK_MTHS']]

### Derive Variables

In [11]:
df_loan_data_basic_var['CUS_AGE_AT_LOAN'] = \
    df_loan_data_basic_var.apply(lambda x: relativedelta(x['DATE_OPEN'], x['BIR_DATE']).years, axis=1)

In [12]:
df_loan_data_basic_var['BANK_YRS'] = \
    df_loan_data_basic_var.apply(lambda x: relativedelta(x['DATE_OPEN'], x['CUS_DATE_OPEN']).years, axis=1)

In [13]:
df_loan_data_basic_var['BANK_MTHS'] = \
    df_loan_data_basic_var.apply(lambda x: (12*x['BANK_YRS']) + relativedelta(x['DATE_OPEN'], x['CUS_DATE_OPEN']).months, axis=1)

In [14]:
# Ratio denominator should not be greater than 12
df_loan_data_basic_var['AIRTIME_NBR_MONTH_ACTIVE_RATIO'] = \
    df_loan_data_basic_var['AIRTIME_NBR_MONTH_ACTIVE']/df_loan_data_basic_var['BANK_MTHS'].apply(lambda x: x if x<12 else 12)

df_loan_data_basic_var['AIRTIME_NBR_MONTH_ACTIVE_RATIO'] = \
    df_loan_data_basic_var['AIRTIME_NBR_MONTH_ACTIVE_RATIO'].apply(lambda x: x if x<=1 else 1)

In [15]:
df_loan_data_basic_var['NTWRK_OUT_IN_DEGREE_RATIO'] = \
    df_loan_data_basic_var['NTWRK_OUT_DEGREE']/df_loan_data_basic_var['NTWRK_IN_DEGREE']

df_loan_data_basic_var['NTWRK_OUT_IN_DEGREE_RATIO'] = \
    df_loan_data_basic_var['NTWRK_OUT_IN_DEGREE_RATIO'].replace(np.inf,np.nan).fillna(0)

In [16]:
# df_loan_data_basic_var['POS_TRA_COUNT_NIGHT_SPEND_RATIO'] = ((df_loan_data_basic_var['POS_TRA_COUNT_0_5'] + 
#                                                               df_loan_data_basic_var['POS_TRA_COUNT_21_23']
#                                                              )/ df_loan_data_basic_var['POS_TRA_COUNT'])

# df_loan_data_basic_var['POS_TRA_COUNT_NIGHT_SPEND_RATIO'] = df_loan_data_basic_var['POS_TRA_COUNT_NIGHT_SPEND_RATIO'].replace(np.inf,np.nan).fillna(0)
# df_loan_data_basic_var.loc[df_loan_data_basic_var['POS_TRA_COUNT_NIGHT_SPEND_RATIO']>=0.2,'IS_POS_NIGHT_SPENDER']=1
# df_loan_data_basic_var['IS_POS_NIGHT_SPENDER'] = df_loan_data_basic_var['IS_POS_NIGHT_SPENDER'].fillna(0)

In [17]:
df_loan_data_basic_var['POS_TRA_DAY_COUNT_NIGHT_SPEND_RATIO'] = \
                                    (df_loan_data_basic_var['POS_TRA_DAY_COUNT_0_5']+   \
                                     df_loan_data_basic_var['POS_TRA_DAY_COUNT_21_23'])/ \
                                    df_loan_data_basic_var['BANK_MTHS']
        
df_loan_data_basic_var.loc[df_loan_data_basic_var['POS_TRA_DAY_COUNT_NIGHT_SPEND_RATIO']>=0.2,'IS_POS_NIGHT_SPENDER']=1
df_loan_data_basic_var['IS_POS_NIGHT_SPENDER'] = df_loan_data_basic_var['IS_POS_NIGHT_SPENDER'].fillna(0)

In [18]:
df_loan_data_basic_var['IB_MB_GTW_COUNT'] = df_loan_data_basic_var['IB_CNT'] + \
                                            df_loan_data_basic_var['MB_CNT'] + \
                                            df_loan_data_basic_var['GTWORLD_CNT']
        
df_loan_data_basic_var['USE_IB_MB_GTW'] = (df_loan_data_basic_var['IB_MB_GTW_COUNT']>0).astype(int)

In [19]:
df_loan_data_basic_var['HAS_IJEBU_LEDGERS']= (df_loan_data_basic_var['NBR_DISTINCT_IJEBU_LEDGERS']>0).astype(int)

In [20]:
df_loan_data_basic_var['HAS_EVER_LOAN']= (df_loan_data_basic_var['NBR_LOANS']>0).astype(int)

In [21]:
df_loan_data_basic_var['HAS_DOM_ACCS']= (df_loan_data_basic_var['NBR_DOM_ACCS']>0).astype(int)

In [22]:
df_loan_data_basic_var['USE_MB_AND_GTW']= \
    ((df_loan_data_basic_var['MB_CNT']>0) & (df_loan_data_basic_var['GTWORLD_CNT']>0)).astype(int)

In [23]:
df_loan_data_basic_var['USE_POS_OR_GAPS']= \
    ((df_loan_data_basic_var['POS_CNT']>0) | (df_loan_data_basic_var['GAPS_CNT']>0)).astype(int)

In [24]:
df_loan_data_basic_var['USE_ATM_ONLY']= \
(((df_loan_data_basic_var['ATM_ROU_CNT']+df_loan_data_basic_var['ATM_ON_US_CNT']==0)>0) & \
 (df_loan_data_basic_var['GTWORLD_CNT']==0) & \
 (df_loan_data_basic_var['IB_CNT']==0) & \
 (df_loan_data_basic_var['POS_WDR_CNT']==0) & \
 (df_loan_data_basic_var['USSD_CNT']==0)).astype(int)

In [25]:
df_loan_data_basic_var['USE_USSD_ONLY']= \
((df_loan_data_basic_var['USSD_CNT']>0) & \
 (df_loan_data_basic_var['ATM_ROU_CNT']==0) & \
 (df_loan_data_basic_var['ATM_ON_US_CNT']==0) & \
 (df_loan_data_basic_var['GTWORLD_CNT']==0) & \
 (df_loan_data_basic_var['IB_CNT']==0) & \
 (df_loan_data_basic_var['POS_WDR_CNT']==0)).astype(int)

In [26]:
df_loan_data_basic_var['HAS_BAD_NEIGHBORS'] = (df_loan_data_basic_var['NBR_BAD_NEIGHBORS']>0).astype(int)

In [27]:
df_loan_data_basic_var['HAS_CONSISTENT_LODGE']= (df_loan_data_basic_var['MONTH_ACTIVE_RATIO']>=0.9).astype(int)

In [28]:
df_loan_data_basic_var['DOES_IN_BRANCH']= (df_loan_data_basic_var['CASH_WDR_CNT']>0).astype(int)

In [29]:
df_loan_data_basic_var['ISSUES_CHEQUE']= (df_loan_data_basic_var['CHEQUES_ISSUED']>0).astype(int)

In [30]:
df_loan_data_basic_var['REACTIVATED_OR_NEW_IN_12_MTHS'] = \
    ((df_loan_data_basic_var['HAS_REACTIVATED_12_MTHS']>0) | (df_loan_data_basic_var['BANK_YRS']==0)).astype(int)

In [31]:
df_loan_data_basic_var['ISSUES_CHEQUE']= (df_loan_data_basic_var['CHEQUES_ISSUED']>0).astype(int)

In [32]:
df_loan_data_basic_var[df_loan_data_basic_var.USE_ATM_ONLY>0].IS_BAD.value_counts()

0.0    1066
1.0     198
Name: IS_BAD, dtype: int64

In [33]:
# df_loan_data_basic_var[df_loan_data_basic_var['HAS_BAD_NEIGHBORS']==1].IS_BAD.value_counts()

In [34]:
# df_loan_data_basic_var[df_loan_data_basic_var['REACTIVATED_OR_NEW_IN_12_MTHS']==1].IS_BAD.value_counts()

In [35]:
# df_loan_data_basic_var[df_loan_data_basic_var['IS_PEP']==1].IS_BAD.value_counts()

In [36]:
# df_loan_data_basic_var[df_loan_data_basic_var['BANK_YRS']==0]

In [37]:
# df_loan_data_basic_var[df_loan_data_basic_var.GAPS_CNT>0].IS_BAD

In [38]:
# df_loan_data_basic_var[df_loan_data_basic_var.POS_CNT>0].IS_BAD

In [39]:
df_loan_data_basic_var[df_loan_data_basic_var.HAS_BIZ_ACC>0].IS_BAD.value_counts()

0.0    4905
1.0     404
Name: IS_BAD, dtype: int64

### Exploration

In [40]:
df_loan_data_basic_var.COV_LODGEMENT_CNT.describe()

count    154724.000000
mean          0.538375
std           0.233406
min           0.000000
25%           0.385276
50%           0.505076
75%           0.648389
max           3.098387
Name: COV_LODGEMENT_CNT, dtype: float64

In [41]:
df_loan_data_basic_var['NTWRK_OUT_IN_DEGREE_RATIO'].describe()
df_loan_data_basic_var['NTWRK_DEGREE'].describe()
df_loan_data_basic_var['AIRTIME_AVG_PER_MONTH_COUNT'].describe()
df_loan_data_basic_var['POS_TRA_DAY_COUNT_NIGHT_SPEND_RATIO'].describe()
df_loan_data_basic_var['SUM_LODGEMENT_AMOUNT'].describe()
print df_loan_data_basic_var['AVG_TURNOVER_AMOUNT'].describe()
print df_loan_data_basic_var['AVG_LODGEMENT_AMOUNT'].describe()
# df_loan_data_basic_var['AIRTIME_NBR_MONTH_ACTIVE_RATIO'].describe()

count    1.547240e+05
mean     3.992692e+05
std      1.426336e+06
min      0.000000e+00
25%      5.604680e+04
50%      1.217163e+05
75%      3.047628e+05
max      1.705020e+08
Name: AVG_TURNOVER_AMOUNT, dtype: float64
count    1.547240e+05
mean     3.983549e+05
std      1.427623e+06
min      9.166667e+02
25%      5.649155e+04
50%      1.220114e+05
75%      3.041263e+05
max      1.705622e+08
Name: AVG_LODGEMENT_AMOUNT, dtype: float64


In [42]:
df_loan_data_basic_var['NTWRK_OUT_IN_DEGREE_RATIO'].describe()
df_loan_data_basic_var['NTWRK_DEGREE'].describe()
df_loan_data_basic_var['AIRTIME_AVG_PER_MONTH_COUNT'].describe()
df_loan_data_basic_var['POS_TRA_DAY_COUNT_NIGHT_SPEND_RATIO'].describe()
df_loan_data_basic_var['SUM_LODGEMENT_AMOUNT'].describe()
df_loan_data_basic_var['AVG_TURNOVER_AMOUNT'].describe()
df_loan_data_basic_var['AIRTIME_NBR_MONTH_ACTIVE_RATIO'].describe()

count    154724.000000
mean          0.755002
std           0.333588
min           0.000000
25%           0.583333
50%           0.916667
75%           1.000000
max           1.000000
Name: AIRTIME_NBR_MONTH_ACTIVE_RATIO, dtype: float64

### Fill missing gender values by Gender Prediction Model

In [43]:
def transform_name(name):
    return loaded_vectorizer.transform([name.upper().strip()])
def genderpredictor(name):
    return loaded_model.predict(transform_name(name))

In [44]:
df_loan_data_basic_var['CUS_NAME'] = df_loan_data_basic_var['CUS_NAME'].astype('str')

In [45]:
# predict gender that is null
df_loan_data_basic_var['CUS_GENDER'] = df_loan_data_basic_var.apply(
    lambda x: genderpredictor(x.CUS_NAME)[0] if not x.CUS_GENDER in ['M','F'] else x.CUS_GENDER, axis=1)

### Binning

In [46]:
bin_args = \
[
 {'col':'CUS_AGE_AT_LOAN', 'bins':[-1,17,29,39,44,999], 'right':True, 'labels':None, 'precision':5},
 {'col':'BANK_YRS', 'bins':[-1,3,5,7,99], 'right':True, 'labels':None, 'precision':5},
 
 {'col':'NBR_DISTINCT_DEP', 'bins':[-1,1,5,99], 'right':True, 'labels':None, 'precision':5},
 {'col':'AIRTIME_NBR_MONTH_ACTIVE_RATIO', 'bins':[-1,0.5,0.8,0.9,1], 'right':True, 'labels':None, 'precision':5},
 {'col':'AIRTIME_AVG_PER_MONTH_COUNT', 'bins':[-1,2,5,10,20,99999], 'right':True, 'labels':None, 'precision':5},
 
 {'col':'NTWRK_OUT_IN_DEGREE_RATIO', 'bins':[-1,0.1,1,2,9999999999999], 'right':True, 'labels':None, 'precision':5},
 {'col':'NTWRK_DEGREE', 'bins':[-1,3,11,30,9999999999999], 'right':True, 'labels':None, 'precision':5},
    
 {'col':'SUM_LODGEMENT_AMOUNT', 'bins':[-1,50000,250000,750000,2000000,9999999999999], 'right':True, 'labels':None, 'precision':5},
 {'col':'AVG_TURNOVER_AMOUNT', 'bins':[-1,3000,6000,10000,9999999999999], 'right':True, 'labels':None, 'precision':5},
 {'col':'AVG_LODGEMENT_AMOUNT', 'bins':[-1,3000,6000,10000,9999999999999], 'right':True, 'labels':None, 'precision':5},
    
 {'col':'COV_LODGEMENT_CNT', 'bins':[-1,0.3,0.5,0.7,1,99999999], 'right':True, 'labels':None, 'precision':5}
]

In [47]:
# Save binning arguments
pickle.dump(bin_args, open('bin_args.sav', 'wb'))

In [49]:
df_loan_data_basic_var =  modeling_utils.binning_dataset(df_loan_data_basic_var, bin_args)

Running for:  CUS_AGE_AT_LOAN
Running for:  BANK_YRS
Running for:  NBR_DISTINCT_DEP
Running for:  AIRTIME_NBR_MONTH_ACTIVE_RATIO
Running for:  AIRTIME_AVG_PER_MONTH_COUNT
Running for:  NTWRK_OUT_IN_DEGREE_RATIO
Running for:  NTWRK_DEGREE
Running for:  SUM_LODGEMENT_AMOUNT
Running for:  AVG_TURNOVER_AMOUNT
Running for:  AVG_LODGEMENT_AMOUNT
Running for:  COV_LODGEMENT_CNT


### Feature Selection

In [50]:
cols_selected_1 = ['CUS_AGE_AT_LOAN_BIN', 'BANK_YRS_BIN', 'NBR_DISTINCT_DEP_BIN', 'AIRTIME_NBR_MONTH_ACTIVE_RATIO_BIN',
                 'AIRTIME_AVG_PER_MONTH_COUNT_BIN', 'NTWRK_OUT_IN_DEGREE_RATIO_BIN', 'NTWRK_DEGREE_BIN', 'IS_BETTOR',
                 'IS_RELIGIOUS', 'IS_VISA', 'IS_PAY_UTILITIES', 'IS_POS_NIGHT_SPENDER', 'CUS_GENDER', 'HAS_REACTIVATED',
                 'AIRTIME_DO_THIRD_PARTY', 'HAS_IJEBU_LEDGERS', 'HAS_EVER_LOAN', 'USE_IB_MB_GTW',
                 'IS_PLATINUM_PLUS_HNI', 'AVG_TURNOVER_AMOUNT_BIN','HAS_BIZ_ACC','REACTIVATED_OR_NEW_IN_12_MTHS',
                 'ISSUES_CHEQUE','DOES_IN_BRANCH','HAS_CONSISTENT_LODGE','HAS_BAD_NEIGHBORS',
                 'HAS_DOM_ACCS','USE_MB_AND_GTW','USE_POS_OR_GAPS','USE_ATM_ONLY','COV_LODGEMENT_CNT_BIN','STATE','PROF_CODE']

In [51]:
pickle.dump(cols_selected_1, open('cols_selected_1.sav', 'wb'))

In [52]:
cols_selected_2_cat = ['CUS_AGE_AT_LOAN_BIN', 'IS_BETTOR', 'CUS_GENDER', 'HAS_REACTIVATED',
                       'IS_RELIGIOUS', 'IS_VISA', 'IS_PAY_UTILITIES', 'IS_POS_NIGHT_SPENDER', 
                       'IS_BETTOR', 'AIRTIME_DO_THIRD_PARTY', 'HAS_IJEBU_LEDGERS', 'HAS_EVER_LOAN',
                       'USE_IB_MB_GTW', 'IS_PLATINUM_PLUS_HNI', 'HAS_BIZ_ACC','REACTIVATED_OR_NEW_IN_12_MTHS', 
                       'ISSUES_CHEQUE','DOES_IN_BRANCH', 'HAS_CONSISTENT_LODGE','HAS_BAD_NEIGHBORS',
                       'HAS_DOM_ACCS','USE_MB_AND_GTW','USE_POS_OR_GAPS','USE_ATM_ONLY','STATE','PROF_CODE'
                      ]

In [53]:
pickle.dump(cols_selected_2_cat, open('cols_selected_2_cat.sav', 'wb'))

In [54]:
cols_selected_2_numeric = [
"BANK_YRS",
"NBR_DISTINCT_DEP", 
"AIRTIME_NBR_MONTH_ACTIVE_RATIO",                
"AIRTIME_AVG_PER_MONTH_COUNT",
"AIRTIME_NBR_MONTH_ACTIVE",
"NBR_CALL_TIME_TBILLS",
"NBR_DISTINCT_IJEBU_LEDGERS",
"NBR_LOANS",
"OUTSTANDING_DEBT",
"DUE_DEBT",
"NBR_DISTINCT_LOAN",
"NBR_DOM_ACCS",
"AVG_BAL_ALL_ACC",
"CRNT_BAL_ALL_ACC",
"SUM_LODGEMENT_AMOUNT",
"SUM_LODGEMENT_COUNT",
"SUM_TURNOVER_AMOUNT",
"SUM_TURNOVER_COUNT",
"STD_DEV_LODGEMENT_AMOUNT",
"STD_DEV_LODGEMENT_COUNT",
"STD_DEV_TURNOVER_AMOUNT",
"STD_DEV_TURNOVER_COUNT",
"MONTHS_ACTIVE_IN_12_MTHS",
"MONTHS_WITH_GTB_IN_12_MTHS",
"AVG_LODGEMENT_AMOUNT",
"AVG_LODGEMENT_COUNT",
"AVG_TURNOVER_AMOUNT",
"AVG_TURNOVER_COUNT",
"IN_OUT_AMT_RATIO",
"IN_OUT_CNT_RATIO",
"COV_TURNOVER_AMT",
"COV_LODGEMENT_AMT",
"COV_TURNOVER_CNT",
"COV_LODGEMENT_CNT",
"POS_TRA_COUNT_0_5",
"POS_TRA_DAY_COUNT_0_5",
"POS_TRA_COUNT_6_11",
"POS_TRA_DAY_COUNT_6_11",
"POS_TRA_COUNT_12_16",
"POS_TRA_DAY_COUNT_12_16",
"POS_TRA_COUNT_17_20",
"POS_TRA_DAY_COUNT_17_20",
"POS_TRA_COUNT_21_23",
"POS_TRA_DAY_COUNT_21_23",
"TRA_AMT_0_5",
"TRA_AMT_6_11",
"TRA_AMT_12_16",
"TRA_AMT_17_20",
"TRA_AMT_21_23",
"TRA_COUNT_0_5",
"TRA_COUNT_6_11",
"TRA_COUNT_12_16",
"TRA_COUNT_17_20",
"TRA_COUNT_21_23",
"NTWRK_OUT_TRA_COUNT",
"NTWRK_OUT_TRA_AMOUNT",
"NTWRK_IN_DEGREE",
"NTWRK_IN_TRA_COUNT",
"NTWRK_IN_TRA_AMOUNT",
"NTWRK_DEGREE",
"NTWRK_TRA_COUNT",
"NTWRK_TRA_AMOUNT",
"NTWRK_OUT_IN_DEGREE_RATIO"
]

In [55]:
pickle.dump(cols_selected_2_numeric, open('cols_selected_2_numeric.sav', 'wb'))

### Check missing values

In [64]:
missing_df_1 = df_loan_data_basic_var[cols_selected_1].isnull().sum()
missing_df_1[missing_df_1>0]

STATE        557
PROF_CODE     78
dtype: int64

In [74]:
missing_df_2 = df_loan_data_basic_var[cols_selected_2_cat].isnull().sum()
missing_df_2[missing_df_2>0]

STATE        557
PROF_CODE     78
dtype: int64

In [75]:
missing_df_3 = df_loan_data_basic_var[cols_selected_2_numeric].isnull().sum()
missing_df_3[missing_df_3>0]

Series([], dtype: int64)

In [73]:
numeric_cols_with_missing_value = missing_df_3[missing_df_3>0].index
df_loan_data_basic_var[numeric_cols_with_missing_value] = df_loan_data_basic_var[numeric_cols_with_missing_value].fillna(0)

### Write output

In [76]:
df_loan_data_basic_var.to_csv('df_loan_data_derived_var.csv', index=False)

In [60]:
# df_loan_data_final_set.to_csv('df_loan_data_final_set.csv', index=False)

In [61]:
# df_loan_data_final_set = pd.read_csv('df_loan_data_final_set.csv')

In [62]:
# df_loan_data_final_set

In [ ]:
# import pprint

In [ ]:
# col_explore = df_loan_data_final_set.columns.drop(['BRA_CODE','CUS_NUM','IS_BAD'])

In [ ]:

# for i, c in enumerate(col_explore):
#     print i+1, '. ', c
#     cur_dfx = pd.pivot_table(df_loan_data_final_set[[c,'IS_BAD','CUS_NUM']].groupby([c,'IS_BAD'], 
#                        as_index=False).agg({'CUS_NUM': 'count'}).rename(columns={'CUS_NUM':'CNT'}),
#                        values='CNT', columns=['IS_BAD'], index=[c], aggfunc=np.sum, fill_value=0)
    
#     categorical_name = cur_dfx.index.name
#     cur_dfx = cur_dfx.append(pd.Series(name=''), ignore_index=True)
#     cur_dfx.columns.name = categorical_name
    
#     file_opt = 'wt'
#     if i>0:
#         file_opt='at'
#     cur_dfx.to_csv('df_explore.csv', mode=file_opt)
